# DREAM instrument view

This notebook is a simple example on how to use the instrument view for the DREAM instrument.

- The DREAM-specific instrument view is capable of slicing the data with a slider widget along a dimension (e.g. `tof`) by using the `dim` argument.
- There are also checkboxes to hide/show the different modules that make up the DREAM detectors.

In [ ]:
import pandas as pd
import scipp as sc
from ess import dream

## Load the data

We load a data set from a Geant4 simulation (stored as a `.csv` file).
In each detector bank, the data is organised by `wire`, `strip`, `module`, `segment`, and `counter`.
The end caps also have an additional `sumo` dimension.

In [ ]:
dg = dream.io.load_geant4_csv(dream.data.get_path('data_dream_HF_mil_closed_alldets_1e9.csv.zip'))
dg = dg['instrument']  # Extract the instrument data

# Construct the pixel positions from event positions
for da in dg.values():
    da.coords['position'] = da.bins.coords['position'].bins.mean()

dg

## Instrument view

We first histogram the data along the time-of-flight (`tof`) dimension,
making sure the same bins are used for all modules:

In [ ]:
tof_edges = sc.linspace('tof', 1.0e7, 1.0e8, 51, unit='ns', dtype=int)
data = dg.hist(tof=tof_edges)

In [ ]:
# Only plot half of the pixels to reduce html docs size
data = data['counter', 0]

We now use the `instrument_view` function to show the 3D view of the instrument pixels,
specifying that we wish to have a slider along the `tof` dimension:

In [ ]:
full_view = dream.instrument_view(data, dim='tof')
full_view

In [ ]:
full_view[2].controls['tof']['slider'].value = 35

Note that it is possible to use any dimension for the slider instead of `tof`, such as `wavelength` (if present in the data).

It is also possible to view a single detector module using

In [ ]:
mantle_view = dream.instrument_view(data['mantle'], dim='tof')
mantle_view

In [ ]:
mantle_view[1].controls['tof']['slider'].value = 35